# MSD 歌曲推荐——数据预处理
1. 计算每个用户的总播放次数，找出播放次数最多的用户（最活跃的前800个用户）
2. 计算每首歌曲的总播放次数，找出播放次数最多的歌曲（最流行的前800首歌曲）
3. 从总数据集中抽取最活跃的前800个用户、最流行的前800首歌曲的播放记录

In [1]:
#coding=utf-8

In [2]:
import pandas as pd
import numpy as np
import time

#Python SQLITE数据库是一款非常小巧的嵌入式开源数据库软件
import sqlite3

## 载入原始数据

### Get more information about the Millionsong project from https://labrosa.ee.columbia.edu/millionsong/

### Load Triplets data  [user, song, play_count]
#### Get the data from http://labrosa.ee.columbia.edu/millionsong/sites/default/files/challenge/train_triplets.txt.zip

In [7]:
#数据集太大，读10000条记录看看
dpath = './data/'
df_triplet_dataset = pd.read_csv(filepath_or_buffer = dpath + 'train_triplets.txt', 
                              nrows=10000,sep='\t', header=None, 
                              names=['user','song','play_count'])

In [8]:
df_triplet_dataset.head(n=10)

,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBNZDC12A6D4FC103,1
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBSUJE12A6D4F8CF5,2
7,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBVFZR12A6D4F8AE3,1
8,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXALG12A8C13C108,1
9,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1


## 计算每个用户的总播放次数 和每首歌曲的播放次数
看哪些用户最活跃（play counts最多）
由于空间有限，不活跃的用户不考虑了（共有约1M用户）
不流行的歌曲也不考虑了

In [16]:
output_dict_users = {} #user及对应的play_count次数集合(在所有的歌曲上求和)
output_dict_songs = {} #song及对应的play_count次数集合(在所有的歌曲上求和)

#也可以用read_csv
with open(dpath + 'train_triplets.txt') as f:
    for line_number, line in enumerate(f):
        cols = line.strip().split("\t")
        user = cols[0]            #第一列为用户id
        song = cols[1]            #第二列为歌曲id
        play_count = int(cols[2]) #第三列为播放次数
        
        if user in output_dict_users:
            play_count += output_dict_users[user]
        output_dict_users.update({user:play_count})
        
        if song in output_dict_songs:
            play_count += output_dict_songs[song]
        output_dict_songs.update({song:play_count})

#用户——播放次数列表
output_list_users = [{'user':k,'play_count':v} for k,v in output_dict_users.items()]
df_users_play_count = pd.DataFrame(output_list_users)

#按总播放次数排序
df_users_play_count = df_users_play_count.sort_values(by = 'play_count', ascending = False)
df_users_play_count.to_csv(path_or_buf= dpath + 'users_playcount.csv', index = False)


#歌曲——播放次数列表
output_list_songs = [{'song':k,'play_count':v} for k,v in output_dict_songs.items()]
df_songs_play_count = pd.DataFrame(output_list_songs)

#按总播放次数排序
df_songs_play_count = df_songs_play_count.sort_values(by = 'play_count', ascending = False)
df_songs_play_count.to_csv(path_or_buf= dpath + 'songs_playcount.csv', index = False)

## 最活跃的用户和最流行的歌曲

In [21]:
df_users_play_count = pd.read_csv(filepath_or_buffer= dpath + 'users_playcount.csv')
df_users_play_count.head(n =10)

,play_count,user
0,13132,093cb74eb3c517c5179ae24caf0ebec51b24d2a2
1,9884,119b7c88d58d0c6eb051365c103da5caf817bea6
2,8210,3fa44653315697f42410a30cb766a4eb102080bb
3,7015,a2679496cd0af9779a92a13ff7c6af5c81ea8c7b
4,6494,d7d2d888ae04d16e994d6964214a1de81392ee04
5,6472,4ae01afa8f2430ea0704d502bc7b57fb52164882
6,6150,b7c24f770be6b802805ac0e2106624a517643c17
7,5656,113255a012b2affeab62607563d03fbdf31b08e7
8,5620,6d625c6557df84b60d90426c0116138b617b9449
9,5602,99ac3d883681e21ea68071019dba828ce76fe94d


In [23]:
df_songs_play_count = pd.read_csv(filepath_or_buffer= dpath + 'songs_playcount.csv')
df_songs_play_count.head(10)

,play_count,song
0,12466637,SOSXLTC12AF72A7F54
1,11290840,SOWCKVR12A8C142411
2,9582939,SOUSMXX12AB0185C24
3,9350709,SONYKOW12AB01849C9
4,9053507,SOUVTSM12AC468F6A7
5,8535928,SOZVCRW12A67ADA0B7
6,7882419,SOXFPND12AB017C9D1
7,7875104,SOTWNDJ12A8C143984
8,7318597,SOPUCYA12A8C13A694
9,7317594,SOXWYZP12AF72A42A6


## 从总体数据中抽取子集
前800个用户
前5000个用户（共1百万用户）的播放次数和占总播放次数的40%（2/8原则：20%的用户播放了80%的次数）

In [27]:
N_SELECTED_USERS = 800
N_SELECTED_ITEMS = 800

df_users_play_count_subset = df_users_play_count.head(n=N_SELECTED_USERS)

n_total_play_count = sum(df_users_play_count.play_count)
slected_percent = (float(df_users_play_count_subset.play_count.sum())/n_total_play_count)*100

print(slected_percent)

1.37758555846


#取800个用户（万分之8的用户）首占总播放次数的1.4%

In [30]:
#共30万首歌曲
df_songs_play_count_subset = df_songs_play_count.head(n=N_SELECTED_ITEMS)

In [31]:
#前800个用户和800首歌曲
users_subset = list(df_users_play_count_subset.user)
songs_subset = list(df_songs_play_count_subset.song)

In [33]:
#如果机器内存不足，可参考前面逐条读txt文件
df_triplet_dataset = pd.read_csv(filepath_or_buffer = dpath + 'train_triplets.txt',sep='\t', 
                              header=None, names=['user','song','play_count'])
df_triplet_dataset_sub_users = df_triplet_dataset[df_triplet_dataset.user.isin(users_subset) ]
del(df_triplet_dataset)

df_triplet_dataset_sub_users_songs = df_triplet_dataset_sub_users[df_triplet_dataset_sub_users.song.isin(songs_subset)]
del(df_triplet_dataset_sub_users)

In [34]:
df_triplet_dataset_sub_users_songs.to_csv(path_or_buf = dpath + 'triplet_dataset_sub.csv', index=False)
df_triplet_dataset_sub_users_songs.shape

(37519, 3)

即使这样，还有4万条记录

In [36]:
df_triplet_dataset_sub_users_songs.head(n=10)

,user,song,play_count
43812,4e11f45d732f4861772b2906f81a7d384552ad12,SOCKSGZ12A58A7CA4B,1
43834,4e11f45d732f4861772b2906f81a7d384552ad12,SOCVTLJ12A6310F0FD,1
43863,4e11f45d732f4861772b2906f81a7d384552ad12,SODLLYS12A8C13A96B,3
43898,4e11f45d732f4861772b2906f81a7d384552ad12,SOEGIYH12A6D4FC0E3,1
43961,4e11f45d732f4861772b2906f81a7d384552ad12,SOFRQTD12A81C233C0,2
44044,4e11f45d732f4861772b2906f81a7d384552ad12,SOHEMBB12A6701E907,1
44057,4e11f45d732f4861772b2906f81a7d384552ad12,SOHJOLH12A6310DFE5,1
44131,4e11f45d732f4861772b2906f81a7d384552ad12,SOIZLKI12A6D4F7B61,1
44146,4e11f45d732f4861772b2906f81a7d384552ad12,SOJGSIO12A8C141DBF,1
44187,4e11f45d732f4861772b2906f81a7d384552ad12,SOKEYJQ12A6D4F6132,1
